In [22]:
!pip install cirq

In [23]:
import cirq
import numpy as np
import random

In [24]:


# --- Configuration Parameters ---
# Number of qubits to transmit. A larger number increases the key length
# but also the chance of errors/eavesdropping detection.
NUM_QUBITS = 100

# Probability of an eavesdropper (Eve) intercepting and re-sending a qubit.
# Set to 0 for no eavesdropping.
EVE_INTERCEPTION_PROB = 0.2

# Probability of a bit flip error in the quantum channel (simulates noise).
# This is independent of Eve's actions.
CHANNEL_NOISE_PROB = 0.05

# --- Qubit Definition ---
# Define a list of qubits for our quantum communication.
# We use LineQubit for simplicity, representing qubits in a line.
qubits = [cirq.LineQubit(i) for i in range(NUM_QUBITS)]


In [25]:

# --- Helper Functions ---

def generate_random_bits(num_bits):
    """Generates a list of random binary bits (0 or 1)."""
    return [random.randint(0, 1) for _ in range(num_bits)]

def generate_random_bases(num_bases):
    """Generates a list of random bases ('Z' for computational, 'X' for Hadamard)."""
    return [random.choice(['Z', 'X']) for _ in range(num_bases)]

def encode_qubit(bit, basis, qubit):
    """
    Encodes a classical bit into a quantum qubit based on the chosen basis.
    - If basis is 'Z' (computational): 0 -> |0>, 1 -> |1>
    - If basis is 'X' (Hadamard): 0 -> |+>, 1 -> |->
    """
    circuit = cirq.Circuit()
    if bit == 1:
        # Apply X gate if bit is 1 to flip |0> to |1>
        circuit.append(cirq.X(qubit))
    if basis == 'X':
        # Apply Hadamard gate if basis is X to transform to superposition states
        circuit.append(cirq.H(qubit))
    return circuit

def measure_qubit(basis, qubit):
    """
    Measures a qubit in the specified basis.
    - If basis is 'Z': measures in computational basis.
    - If basis is 'X': applies Hadamard, then measures in computational basis.
    """
    circuit = cirq.Circuit()
    if basis == 'X':
        # Apply Hadamard gate to transform from Hadamard to computational basis for measurement
        circuit.append(cirq.H(qubit))
    # Measure the qubit and store the result
    circuit.append(cirq.measure(qubit, key=f'm_{qubit.x}'))
    return circuit

def apply_channel_noise(qubit, noise_prob):
    """
    Applies a bit-flip error channel to a qubit with a given probability.
    This simulates environmental noise in the quantum channel.
    """
    # cirq.depolarize(p) applies a depolarizing channel, which is a more general
    # noise model. For a simple bit flip, we can use cirq.bit_flip(p).
    # Here, we'll use bit_flip for simplicity as it's easier to conceptualize
    # as a direct error on the bit.
    return cirq.bit_flip(noise_prob).on(qubit)


In [26]:
# --- BB84 Protocol Simulation ---

def bb84_protocol_simulation(num_qubits, eve_prob, noise_prob):
    """
    Simulates the BB84 Quantum Key Distribution protocol.
    """
    print(f"--- BB84 QKD Simulation ({num_qubits} qubits) ---")
    print(f"Eve's Interception Probability: {eve_prob*100}%")
    print(f"Quantum Channel Noise Probability: {noise_prob*100}%\n")

    # 1. Alice's Preparation
    # Alice generates random bits she wants to send (her raw key).
    alice_bits = generate_random_bits(num_qubits)
    # Alice generates random bases for each bit.
    alice_bases = generate_random_bases(num_qubits)

    print("Alice's initial bits:", alice_bits)
    print("Alice's chosen bases:", alice_bases)

    # List to hold the quantum circuits for each qubit transmission
    alice_circuits = []
    for i in range(num_qubits):
        # Alice encodes each bit into a qubit based on her chosen basis.
        alice_circuits.append(encode_qubit(alice_bits[i], alice_bases[i], qubits[i]))

    # 2. Quantum Channel (Alice sends qubits to Bob)
    # This is where Eve might intercept or noise might occur.
    sim = cirq.Simulator()
    bob_received_qubits_states = [] # To store the state vectors if Eve doesn't intercept
    eve_intercepted_qubits_states = [] # To store the state vectors if Eve intercepts

    # Simulate the quantum channel, including potential eavesdropping and noise.
    for i in range(num_qubits):
        current_circuit = cirq.Circuit()
        current_circuit.append(alice_circuits[i]) # Alice's encoding

        # Simulate channel noise
        if noise_prob > 0:
            current_circuit.append(apply_channel_noise(qubits[i], noise_prob))

        # Simulate Eve's interception
        eavesdropped = random.random() < eve_prob
        if eavesdropped:
            # Eve randomly chooses a basis to measure
            eve_basis = random.choice(['Z', 'X'])
            # Eve measures the qubit
            eve_measurement_circuit = measure_qubit(eve_basis, qubits[i])
            eve_result = sim.run(eve_measurement_circuit, repetitions=1)
            eve_measured_bit = eve_result.measurements[f'm_{qubits[i].x}'][0][0]
            eve_intercepted_qubits_states.append((eve_measured_bit, eve_basis))

            # Eve re-prepares the qubit based on her measurement and sends to Bob
            # This is where the no-cloning theorem comes into play: Eve doesn't
            # know Alice's original basis, so her re-preparation might be wrong.
            re_prepared_circuit = encode_qubit(eve_measured_bit, eve_basis, qubits[i])
            current_circuit.append(re_prepared_circuit)
            bob_received_qubits_states.append(None) # Indicate Eve intercepted
            print(f"  Qubit {i}: Eve intercepted (measured in {eve_basis} basis, got {eve_measured_bit})")
        else:
            # If Eve doesn't intercept, Bob receives the qubit as sent by Alice
            # We'll run Alice's circuit to get the final state before Bob measures
            # For a real QKD, Bob directly receives the quantum state. Here, we
            # simulate passing the state along.
            # We need to simulate the circuit up to this point to get the state for Bob.
            # For simplicity, we'll just indicate it was not intercepted.
            bob_received_qubits_states.append(current_circuit)
            eve_intercepted_qubits_states.append(None) # Indicate Eve did not intercept
            print(f"  Qubit {i}: Not intercepted by Eve.")

    # 3. Bob's Measurement
    # Bob generates random bases for his measurements.
    bob_bases = generate_random_bases(num_qubits)
    print("\nBob's chosen bases:", bob_bases)

    bob_measured_bits = []
    for i in range(num_qubits):
        if bob_received_qubits_states[i] is not None:
            # Bob measures the qubit he received
            bob_measurement_circuit = bob_received_qubits_states[i] + measure_qubit(bob_bases[i], qubits[i])
            bob_result = sim.run(bob_measurement_circuit, repetitions=1)
            bob_measured_bits.append(bob_result.measurements[f'm_{qubits[i].x}'][0][0])
        elif eve_intercepted_qubits_states[i] is not None:
            # If Eve intercepted, Bob measures the qubit re-prepared by Eve.
            # We need to create a circuit for Eve's re-prepared state and Bob's measurement.
            eve_bit, eve_basis = eve_intercepted_qubits_states[i]
            eve_re_prepared_circuit = encode_qubit(eve_bit, eve_basis, qubits[i])
            bob_measurement_circuit = eve_re_prepared_circuit + measure_qubit(bob_bases[i], qubits[i])
            bob_result = sim.run(bob_measurement_circuit, repetitions=1)
            bob_measured_bits.append(bob_result.measurements[f'm_{qubits[i].x}'][0][0])
        else:
            # This case should ideally not happen if logic is correct
            bob_measured_bits.append(None)

    print("Bob's measured bits:", bob_measured_bits)

    # 4. Classical Communication and Sifting
    # Alice and Bob publicly compare their chosen bases.
    # They keep only the bits where their bases matched.
    sifted_alice_key = []
    sifted_bob_key = []
    mismatched_bases_count = 0
    potential_eavesdropping_errors = 0

    print("\n--- Sifting Phase ---")
    for i in range(num_qubits):
        if alice_bases[i] == bob_bases[i]:
            # Bases match, keep the bit
            sifted_alice_key.append(alice_bits[i])
            sifted_bob_key.append(bob_measured_bits[i])
            print(f"  Qubit {i}: Bases match ({alice_bases[i]}). Alice: {alice_bits[i]}, Bob: {bob_measured_bits[i]}")
            # Check for errors in the sifted key (due to noise or Eve)
            if alice_bits[i] != bob_measured_bits[i]:
                potential_eavesdropping_errors += 1
                print(f"    -> ERROR detected in sifted key at Qubit {i}!")
        else:
            # Bases don't match, discard the bit
            mismatched_bases_count += 1
            print(f"  Qubit {i}: Bases mismatch (Alice: {alice_bases[i]}, Bob: {bob_bases[i]}). Discarded.")

    print(f"\nTotal qubits: {num_qubits}")
    print(f"Bases mismatched and discarded: {mismatched_bases_count}")
    print(f"Sifted key length: {len(sifted_alice_key)}")
    print("Alice's sifted key:", sifted_alice_key)
    print("Bob's sifted key:", sifted_bob_key)

    # 5. Error Checking and Privacy Amplification (Simplified)
    # Alice and Bob publicly compare a subset of their sifted key bits
    # to estimate the error rate and detect eavesdropping.
    # If the error rate is too high, they abort the key exchange.
    # This is a simplified check. In reality, more sophisticated error
    # correction and privacy amplification protocols are used.

    if len(sifted_alice_key) == 0:
        print("\nNo sifted key generated. Aborting.")
        return

    # Calculate the bit error rate (QBER - Quantum Bit Error Rate)
    # This is based on the bits where bases matched.
    if len(sifted_alice_key) > 0:
        actual_errors = sum(1 for a_bit, b_bit in zip(sifted_alice_key, sifted_bob_key) if a_bit != b_bit)
        qber = actual_errors / len(sifted_alice_key)
        print(f"\n--- Error Checking ---")
        print(f"Actual errors in sifted key (Alice vs Bob): {actual_errors}")
        print(f"Quantum Bit Error Rate (QBER): {qber:.2f}")

        # A common threshold for QBER to detect eavesdropping is around 11%
        # (due to the 25% chance of Eve introducing an error when bases mismatch).
        # However, noise also contributes.
        if qber > 0.15: # Example threshold, can be adjusted
            print("\n!!! HIGH QBER DETECTED! Potential EAVESDROPPING or high channel noise. Aborting key exchange. !!!")
        else:
            print("\nQBER is within acceptable limits. Key exchange can proceed.")
            print("The sifted key can now be used as a shared secret key.")
            print("Further steps (error correction, privacy amplification) would be applied in a real scenario.")
    else:
        print("\nSifted key is empty, cannot calculate QBER.")

    print("\n--- End of BB84 Simulation ---")

# --- Run the simulation ---
if __name__ == "__main__":
    bb84_protocol_simulation(NUM_QUBITS, EVE_INTERCEPTION_PROB, CHANNEL_NOISE_PROB)

--- BB84 QKD Simulation (100 qubits) ---
Eve's Interception Probability: 20.0%
Quantum Channel Noise Probability: 5.0%

Alice's initial bits: [1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1]
Alice's chosen bases: ['Z', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'X', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'X', 'Z', 'X', 'X', 'Z', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'X', 'Z', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'X', 'X', 'X', 'X', 'X', 'X', 'Z', 'X', 'X', 'X', 'Z', 'X', 'X', 'X', 'Z', 'X', 'X', 'X', 'Z', 'X', 'X', 'Z', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'X']
  Qubit 0: Not intercepted by Eve.